In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from Activity import Activity
from activityByProduct import activityByProduct
from pyspark.streaming.kafka import KafkaUtils

In [2]:
sc = SparkContext(appName="PythonSparkStreamingKafka")
sc.setLogLevel("WARN")

In [3]:
sqlcontext = SQLContext(sc)

In [4]:
ssc = StreamingContext(sc,2)

In [5]:
textDStream1 = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'data_pipeline':1})
textDStream = textDStream1.map(lambda x: x[1])

In [6]:
def subprocess(line):
    records = line.split("\t")
    if records[0] != "action" and len(records) == 7:
        return Activity(records[0],records[1],records[2],records[3],records[4],records[5],records[6])
    else:
        return None

In [7]:
def process(rdd):
    newRDD = rdd.map(subprocess)
    return newRDD

In [8]:
def process_subquery(line):
    return (line[0],line[1]),activityByProduct(line[0],line[1],line[2],line[3],line[4])

In [9]:
def process_query(rdd):
    newDF=rdd.map(lambda x: (x, )).toDF()
    newDF2 = newDF.select('_1.*')
    newDF2.createOrReplaceTempView("activity")
    toHDFS = sqlcontext.sql("select * from activity")
    toHDFS.write.mode("append").csv("hdfs://localhost:9000/kafka_spark",header=True)
    
    activityByProduct = sqlcontext.sql("SELECT product, timestamp_hour, sum(case when action = 'purchase' then 1 else 0 end) as purchase_count, sum(case when action = 'add_to_cart' then 1 else 0 end) as add_to_cart_count, sum(case when action = 'page_view' then 1 else 0 end) as page_view_count from activity group by product, timestamp_hour")
    #activityByProduct.createOrReplaceTempView("activityByProduct")
    activityByProduct1 = activityByProduct.rdd
    finalRDD =activityByProduct1.map(process_subquery)
    return finalRDD 

In [10]:
newRdd = textDStream.transform(process)

In [11]:
newRdd.foreachRDD(process_query)

In [12]:
ssc.start()

In [13]:
# ssc.awaitTermination()